In [1]:
pip install selenium pillow

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/ea/37/d07ed9d13e571b2115d4ed6956d156c66816ceec0b03b2e463e80d09f572/selenium-4.32.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/69/8e/3f6dfda475ecd940e786defe6df6c500734e686c9cd0a0f8ef6821e9b2f2/trio-0.30.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/c7/19/eb640a397bba49ba49ef9dbe2e7e5c04202ba045b6ce2ec36e9cadc51e04/trio_websocket-0.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for typing_extensions~=4.9 from https://files.pythonhosted.org/packages/8b/54/b1ae86c0973cc6f0210b53d508ca3641fb6d0c56823f288d108bc7ab3cc8/typing_extensions-4.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for websocket-client~=1.8 from https://files.pythonhosted.org/packages/5a/84/44687a29792a70e111c5c477230a72c4b957d88d16

In [11]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
from io import BytesIO

# --- Configurable Variables ---
SEARCH_QUERY = "Cryptanthus bivittatus"
DOWNLOAD_DIR = "./cryptanthus-cryptanthus_bivittatus"
NUM_IMAGES = 300
HEADLESS = True
# ------------------------------

def create_driver(headless=True):
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--log-level=3")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def scroll_and_click_see_more(driver):
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        try:
            see_more_button = WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a.btn_seemore"))
            )
            print("Clicking 'See more images' button...")
            ActionChains(driver).move_to_element(see_more_button).click().perform()
            time.sleep(2)
        except:
            # Button not found or not clickable
            break

def collect_thumbnails(driver, max_images):
    thumbnails = set()
    while len(thumbnails) < max_images:
        scroll_and_click_see_more(driver)
        new_thumbs = driver.find_elements(By.CSS_SELECTOR, "img.mimg")
        thumbnails.update(new_thumbs)
        if len(new_thumbs) == 0 or len(thumbnails) >= max_images:
            break
    return list(thumbnails)[:max_images]

def download_images(thumbnails, download_path):
    os.makedirs(download_path, exist_ok=True)
    count = 0
    for i, thumb in enumerate(thumbnails):
        try:
            src = thumb.get_attribute("src")
            if src and src.startswith("http"):
                response = requests.get(src, timeout=10)
                img = Image.open(BytesIO(response.content)).convert("RGB")
                filename = f"IMG_{count+1:04}.jpg"
                img.save(os.path.join(download_path, filename), "JPEG")
                count += 1
                print(f"Downloaded {filename}")
        except Exception as e:
            print(f"Failed to download image {i}: {e}")
    print(f"Downloaded {count} images.")

def main():
    driver = create_driver(headless=HEADLESS)
    query_url = f"https://www.bing.com/images/search?q={SEARCH_QUERY.replace(' ', '+')}"
    driver.get(query_url)
    time.sleep(2)

    print("Collecting thumbnails...")
    thumbnails = collect_thumbnails(driver, NUM_IMAGES)

    print(f"Collected {len(thumbnails)} image thumbnails.")
    download_images(thumbnails, DOWNLOAD_DIR)

    driver.quit()

if __name__ == "__main__":
    main()


Clicking 'See more images' button...
Collected 300 image thumbnails.
Downloaded IMG_0001.jpg
Downloaded IMG_0002.jpg
Downloaded IMG_0003.jpg
Downloaded IMG_0004.jpg
Downloaded IMG_0005.jpg
Downloaded IMG_0006.jpg
Downloaded IMG_0007.jpg
Downloaded IMG_0008.jpg
Downloaded IMG_0009.jpg
Downloaded IMG_0010.jpg
Downloaded IMG_0011.jpg
Downloaded IMG_0012.jpg
Downloaded IMG_0013.jpg
Downloaded IMG_0014.jpg
Downloaded IMG_0015.jpg
Downloaded IMG_0016.jpg
Downloaded IMG_0017.jpg
Downloaded IMG_0018.jpg
Downloaded IMG_0019.jpg
Downloaded IMG_0020.jpg
Downloaded IMG_0021.jpg
Downloaded IMG_0022.jpg
Downloaded IMG_0023.jpg
Downloaded IMG_0024.jpg
Downloaded IMG_0025.jpg
Downloaded IMG_0026.jpg
Downloaded IMG_0027.jpg
Downloaded IMG_0028.jpg
Downloaded IMG_0029.jpg
Downloaded IMG_0030.jpg
Downloaded IMG_0031.jpg
Downloaded IMG_0032.jpg
Downloaded IMG_0033.jpg
Downloaded IMG_0034.jpg
Downloaded IMG_0035.jpg
Downloaded IMG_0036.jpg
Downloaded IMG_0037.jpg
Downloaded IMG_0038.jpg
Downloaded IMG_0039

In [ ]:
main()